# Chapter 6: Classes and Objects

Welcome to the heart of C# programming! So far, we’ve been working with primitive data types, collections, and methods. But real‑world applications are built by modeling real‑world entities: a customer, an invoice, a bank account, a game character. In C#, we model these entities using **classes** and **objects**.

A **class** is a blueprint or template that defines the structure and behavior (data and methods) that the objects of that class will have. An **object** is an instance of a class – a concrete entity created from that blueprint.

In this chapter, you’ll learn:

- How to define a class with **fields**, **properties**, and **methods**.
- How to create objects using the `new` keyword.
- The difference between **static** and instance members.
- How to control access to class members with **access modifiers** (`public`, `private`, `internal`, `protected`).
- The role of the `this` keyword.

Understanding these concepts is the foundation of **object‑oriented programming (OOP)**, which helps you write code that is modular, reusable, and easier to maintain.

---

## 6.1 Defining a Class: Fields, Properties, and Methods

A class is defined using the `class` keyword followed by a name (by convention, PascalCase) and a pair of curly braces containing its members.

### A Simple Class

```csharp
public class Person
{
    // Fields (data)
    public string Name;
    public int Age;

    // Method (behavior)
    public void Introduce()
    {
        Console.WriteLine($"Hi, I'm {Name} and I'm {Age} years old.");
    }
}
```

This `Person` class has two **fields** (`Name` and `Age`) and one **method** (`Introduce`). Fields store the state of an object; methods define its behavior.

### Fields

Fields are variables declared directly inside a class. They hold the data for an object. By convention, fields are usually `private` (we’ll discuss access modifiers soon) and are named with camelCase. If they are `public`, they are named with PascalCase, but exposing public fields is generally discouraged because it breaks encapsulation. Instead, we use **properties**.

### Properties

Properties are members that provide a flexible mechanism to read, write, or compute the values of private fields. They look like fields from the outside but internally can contain logic.

#### Auto‑Implemented Properties

The simplest form is an **auto‑implemented property**, where the compiler generates a hidden backing field for you.

```csharp
public class Person
{
    public string Name { get; set; }
    public int Age { get; set; }
}
```

Here, `Name` and `Age` are properties. You can get and set their values just like fields, but behind the scenes they use accessors (`get` and `set`).

#### Full Property Syntax

If you need validation or additional logic, you can provide a full property implementation with an explicit backing field.

```csharp
public class Person
{
    private string _name;
    private int _age;

    public string Name
    {
        get { return _name; }
        set 
        { 
            if (string.IsNullOrWhiteSpace(value))
                throw new ArgumentException("Name cannot be empty");
            _name = value; 
        }
    }

    public int Age
    {
        get { return _age; }
        set 
        { 
            if (value < 0 || value > 150)
                throw new ArgumentException("Age must be between 0 and 150");
            _age = value; 
        }
    }
}
```

Now you have control over what values are assigned.

#### Read‑Only and Write‑Only Properties

You can omit the `set` accessor to create a read‑only property, or omit the `get` for a write‑only property (though write‑only is rare).

```csharp
public class Person
{
    private DateTime _birthDate;

    public Person(DateTime birthDate)
    {
        _birthDate = birthDate;
    }

    public int Age
    {
        get
        {
            int age = DateTime.Now.Year - _birthDate.Year;
            if (DateTime.Now < _birthDate.AddYears(age)) age--;
            return age;
        }
    }
}
```

Here `Age` is computed from `_birthDate` and has no `set` – it’s read‑only.

### Methods

Methods define the actions an object can perform. They can use the object’s fields and properties.

```csharp
public class Person
{
    public string Name { get; set; }
    public int Age { get; set; }

    public void Introduce()
    {
        Console.WriteLine($"Hi, I'm {Name} and I'm {Age} years old.");
    }

    public bool IsAdult()
    {
        return Age >= 18;
    }
}
```

Methods can also take parameters and return values, as you learned in Chapter 5.

---

## 6.2 Objects and Instances: The `new` Keyword

A class is just a blueprint. To create an actual object (an instance) that exists in memory, you use the `new` keyword.

```csharp
Person person1 = new Person();  // create a new Person object
person1.Name = "Alice";
person1.Age = 30;
person1.Introduce();            // calls the method on that object
```

You can create multiple independent instances:

```csharp
Person person2 = new Person();
person2.Name = "Bob";
person2.Age = 25;
person2.Introduce();
```

Each object has its own copy of the instance fields, so changing `person1.Name` does not affect `person2.Name`.

### Object Initializers

C# provides a concise syntax to set properties at the time of creation using **object initializers**:

```csharp
Person person = new Person { Name = "Charlie", Age = 35 };
```

This is equivalent to creating the object and then assigning the properties, but it’s more readable and can be used in expressions.

### The `new` Keyword and Constructors

When you write `new Person()`, you’re calling a special method called a **constructor**. If you don’t define any constructors, the compiler provides a default parameterless constructor that initializes fields to their default values (e.g., `null` for reference types, `0` for numeric types). We’ll dive deep into constructors in Chapter 8, but for now, know that you can also define your own constructors to require certain values when creating an object.

```csharp
public class Person
{
    public string Name { get; set; }
    public int Age { get; set; }

    // Custom constructor
    public Person(string name, int age)
    {
        Name = name;
        Age = age;
    }
}

// Usage
Person person = new Person("Diana", 28);
```

With a custom constructor, you can no longer use the default parameterless constructor unless you explicitly add one.

---

## 6.3 The `static` Keyword

Not all members belong to an instance. Some members belong to the **class itself** – they are shared across all instances. These are marked with the `static` keyword.

### Static Fields

A static field holds a value that is shared by all objects of the class.

```csharp
public class Counter
{
    public static int TotalCount = 0;  // shared counter

    public Counter()
    {
        TotalCount++;  // increment whenever a new instance is created
    }
}

// Usage
Counter c1 = new Counter();
Counter c2 = new Counter();
Console.WriteLine(Counter.TotalCount); // 2
```

Static fields are accessed using the class name, not an instance variable.

### Static Methods

A static method can be called without creating an instance. It cannot access instance members directly (because there is no `this` reference), but it can access other static members.

```csharp
public class MathUtilities
{
    public static int Add(int a, int b)
    {
        return a + b;
    }
}

int result = MathUtilities.Add(5, 3); // called on the class, not an object
```

You’ve already used many static methods, like `Console.WriteLine` and `int.Parse`.

### Static Classes

A class can be declared `static`. Such a class cannot be instantiated, and can only contain static members. It’s typically used for utility classes.

```csharp
public static class FileHelper
{
    public static void DeleteFile(string path)
    {
        // implementation
    }
}
```

You cannot create an instance of `FileHelper`; you just call its static methods.

### `using static` Directive

C# 6 introduced the `using static` directive, which allows you to import the static members of a class so you can use them without the class name.

```csharp
using static System.Console;

public class Program
{
    public static void Main()
    {
        WriteLine("Hello, World!");  // instead of Console.WriteLine
    }
}
```

This can make code more concise, but use it judiciously to avoid confusion.

---

## 6.4 Access Modifiers: Controlling Visibility

Access modifiers determine which parts of your code can see and use a class member. They are fundamental to **encapsulation** – hiding internal details and exposing only what’s necessary.

C# provides several access modifiers:

| Modifier      | Description |
|---------------|-------------|
| `public`      | Accessible from any code. |
| `private`     | Accessible only within the same class. |
| `internal`    | Accessible only within the same assembly (project). |
| `protected`   | Accessible within the same class and derived classes. |
| `protected internal` | Accessible within the same assembly or derived classes. |
| `private protected` | Accessible within the same class or derived classes, but only if in the same assembly. |

### Applying Access Modifiers

By default, class members are `private` if you don’t specify an access modifier.

```csharp
public class BankAccount
{
    private decimal _balance;           // only accessible inside this class

    public decimal Balance               // public property to expose balance safely
    {
        get { return _balance; }
    }

    public void Deposit(decimal amount)
    {
        if (amount <= 0)
            throw new ArgumentException("Amount must be positive");
        _balance += amount;
    }

    public void Withdraw(decimal amount)
    {
        if (amount <= 0)
            throw new ArgumentException("Amount must be positive");
        if (amount > _balance)
            throw new InvalidOperationException("Insufficient funds");
        _balance -= amount;
    }
}
```

Here, `_balance` is `private` – it cannot be directly changed from outside. The public methods `Deposit` and `Withdraw` enforce business rules, and the public property `Balance` allows read‑only access.

### Why Encapsulation?

Encapsulation protects the integrity of an object’s data. By hiding internal fields and providing controlled access through methods and properties, you can:

- Validate data before assigning.
- Change internal implementation without affecting external code.
- Ensure the object is always in a valid state.

### Access Modifiers for Classes

Top‑level classes (not nested) can only be `public` or `internal`. By default, they are `internal`. If you want a class to be usable from other assemblies, you mark it `public`.

---

## 6.5 The `this` Keyword

Inside an instance method or property, the `this` keyword refers to the current object. It’s used to:

- Distinguish between class members and parameters with the same name.
- Pass the current object as a parameter to another method.
- Call another constructor from a constructor (constructor chaining – we’ll see in Chapter 8).

```csharp
public class Person
{
    private string name;

    public Person(string name)
    {
        this.name = name;  // this.name refers to the field, name is the parameter
    }

    public void Print()
    {
        Console.WriteLine(this.name); // 'this' is optional here
    }

    public Person GetOlder()
    {
        // return this object (allows method chaining)
        return this;
    }
}
```

While `this` is often optional, using it can make code clearer, especially in constructors and when naming conflicts exist.

---

## 6.6 Putting It All Together: A Practical Example

Let’s build a simple `BankAccount` class that demonstrates the concepts we’ve covered: fields, properties, methods, constructors, static members, and access modifiers.

```csharp
using System;
using System.Collections.Generic;

namespace BankSystem
{
    public class BankAccount
    {
        // Static field to generate unique account numbers
        private static int _lastAccountNumber = 1000;

        // Instance fields (private)
        private readonly string _accountNumber;
        private decimal _balance;
        private List<string> _transactionHistory;

        // Public properties
        public string AccountNumber => _accountNumber; // read-only
        public decimal Balance => _balance;             // read-only
        public IReadOnlyList<string> TransactionHistory => _transactionHistory.AsReadOnly();

        // Constructor
        public BankAccount(string ownerName, decimal initialDeposit)
        {
            if (string.IsNullOrWhiteSpace(ownerName))
                throw new ArgumentException("Owner name is required");
            if (initialDeposit < 0)
                throw new ArgumentException("Initial deposit cannot be negative");

            OwnerName = ownerName;
            _accountNumber = GenerateAccountNumber();
            _balance = initialDeposit;
            _transactionHistory = new List<string>();
            AddTransaction($"Account created with initial deposit {initialDeposit:C}");
        }

        // Auto-implemented property (public get, private set)
        public string OwnerName { get; private set; }

        // Private static method to generate a unique account number
        private static string GenerateAccountNumber()
        {
            _lastAccountNumber++;
            return "ACC" + _lastAccountNumber.ToString("D6");
        }

        // Public methods
        public void Deposit(decimal amount)
        {
            if (amount <= 0)
                throw new ArgumentException("Deposit amount must be positive");
            _balance += amount;
            AddTransaction($"Deposited {amount:C}");
        }

        public void Withdraw(decimal amount)
        {
            if (amount <= 0)
                throw new ArgumentException("Withdrawal amount must be positive");
            if (amount > _balance)
                throw new InvalidOperationException("Insufficient funds");
            _balance -= amount;
            AddTransaction($"Withdrew {amount:C}");
        }

        // Private helper method
        private void AddTransaction(string description)
        {
            string entry = $"{DateTime.Now:yyyy-MM-dd HH:mm:ss} - {description}";
            _transactionHistory.Add(entry);
        }

        // Static method to demonstrate utility
        public static void DisplayBankInfo()
        {
            Console.WriteLine($"Welcome to Our Bank");
            Console.WriteLine($"Last account number generated: {_lastAccountNumber}");
        }
    }

    class Program
    {
        static void Main()
        {
            // Create accounts
            BankAccount acc1 = new BankAccount("Alice", 1000m);
            BankAccount acc2 = new BankAccount("Bob", 500m);

            // Perform transactions
            acc1.Deposit(200m);
            acc1.Withdraw(50m);
            acc2.Withdraw(100m);

            // Display account info
            Console.WriteLine($"Account {acc1.AccountNumber} - {acc1.OwnerName}: Balance = {acc1.Balance:C}");
            foreach (string transaction in acc1.TransactionHistory)
            {
                Console.WriteLine($"  {transaction}");
            }

            Console.WriteLine();

            Console.WriteLine($"Account {acc2.AccountNumber} - {acc2.OwnerName}: Balance = {acc2.Balance:C}");
            foreach (string transaction in acc2.TransactionHistory)
            {
                Console.WriteLine($"  {transaction}");
            }

            // Static method call
            BankAccount.DisplayBankInfo();
        }
    }
}
```

**Explanation:**

- **Static field** `_lastAccountNumber` is shared across all instances, used to generate unique account numbers.
- **Private fields** `_accountNumber`, `_balance`, `_transactionHistory` hold internal state.
- **Read‑only properties** expose account number and balance safely. The transaction history is exposed as `IReadOnlyList<string>` to prevent external modification.
- **Constructor** validates parameters and initializes fields. It calls a static method to generate an account number.
- **Public methods** `Deposit` and `Withdraw` change the balance and add transaction records. They contain business rules (no negative amounts, sufficient funds).
- **Private helper method** `AddTransaction` encapsulates the logic of adding a transaction entry.
- **Static method** `DisplayBankInfo` shows bank‑level information.

This example demonstrates real‑world encapsulation: the internal data is protected, and the object’s behavior is controlled through well‑defined methods.

---

## 6.7 Common Pitfalls and Best Practices

### 1. Prefer Properties Over Public Fields

Public fields break encapsulation because you cannot add validation or change the internal representation later. Always use properties (auto‑implemented or full) for public data.

### 2. Keep Fields Private

Fields should almost always be `private`. If you need to expose data, use a property. If you need to expose a field to derived classes, consider `protected`.

### 3. Use Auto‑Implemented Properties for Simple Data

When you don’t need validation logic, auto‑implemented properties are concise and clear. You can later change them to full properties without breaking callers.

### 4. Initialize Fields and Properties

Ensure that objects are created in a valid state. Use constructors to enforce required data. For optional data, you can use object initializers or set defaults.

### 5. Understand When to Use `static`

Static members are useful for utility methods, factory methods, or shared data. However, overusing static can make code hard to test and inflexible. Use static judiciously.

### 6. Use `readonly` for Immutable Fields

If a field should be set only once (in the constructor) and never changed, mark it `readonly`. This expresses intent and prevents accidental modification.

```csharp
public class Person
{
    private readonly DateTime _birthDate;

    public Person(DateTime birthDate)
    {
        _birthDate = birthDate;
    }
}
```

### 7. Follow Naming Conventions

- Classes: PascalCase (e.g., `BankAccount`)
- Public members (fields, properties, methods): PascalCase (e.g., `Deposit`)
- Private fields: `_camelCase` (e.g., `_balance`)
- Parameters and local variables: camelCase (e.g., `amount`)

Consistent naming makes code more readable.

### 8. Keep Classes Focused

A class should have a single responsibility. If a class does too much, consider splitting it into smaller, more focused classes. This is the **Single Responsibility Principle**.

### 9. Use Expression‑Bodied Members Where Appropriate

Simple methods and properties can be written concisely with `=>`. For example:

```csharp
public string FullName => $"{FirstName} {LastName}";
```

---

## 6.8 Chapter Summary

In this chapter, you’ve taken your first major step into object‑oriented programming with C#:

- You learned how to define a class with **fields**, **properties**, and **methods**.
- You created objects using the `new` keyword and object initializers.
- You understood the difference between **static** and instance members, and how to use static classes and the `using static` directive.
- You explored **access modifiers** to control visibility and enforce encapsulation.
- You saw how the `this` keyword refers to the current object.
- You examined a complete, practical example that brings all these concepts together.

Classes and objects are the foundation of virtually all C# applications. Mastering them is essential for building well‑structured, maintainable software.

In the next chapter, **Properties, Indexers, and Records**, we’ll dive deeper into properties (including computed properties), learn about indexers that let your objects behave like arrays, and explore **records** – a special kind of class introduced in C# 9 for immutable data objects. You’ll see how these features make your code more expressive and robust.

**Exercises:**

1. Define a class `Book` with properties `Title`, `Author`, `ISBN`, and `Price`. Ensure `Price` cannot be negative. Add a method `GetDisplayText()` that returns a formatted string.
2. Create a `Library` class that contains a list of `Book` objects. Add methods to add a book, remove a book by ISBN, and find books by author.
3. Add a static field to `Book` that counts how many book objects have been created. Expose it via a static property.
4. Create a `Temperature` class that stores the temperature in Celsius internally, but provides properties to get and set in Fahrenheit and Kelvin (with conversion logic).
5. Experiment with access modifiers: create a class with a mix of `public`, `private`, `protected` members and try to access them from another class in the same project and from a derived class.

Now, get ready to enhance your classes with more powerful features in Chapter 7!

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../1. the_foundation_core_concepts/5. methods_the_building_blocks_of_logic.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='7. properties_indexers_and_records.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
